### M&uuml;hle, Skizze

M&uuml;hle-Koordinaten: `(r, i)`, wobei
- `r` ist einer der Ringe 0, 1 ,2. 0 ist der innerste Ring.
- `i` ist einer der Punkte auf diesem Ring, nummeriert z.B.  
$\begin{array}{ccc}
6& 5& 4\\
7 &&  3\\
0 &1 &2\\
\end{array}$  


Man kann von `(r,i)` nach  `(s,j)` ziehen, falls
- `r == s and abs(i - j) in (1, 7)` (benachbart im gleichen Ring)
- `abs(r - s) == 1 and i == j and is_odd(i)` (benachbart auf Verbindung zw. Ringen)

M&uuml;hlen (falls $8\equiv 0$)
- $[(r, i), (r, i+1), (r, i+2)]$ ist Ring-M&uuml;hle, falls $r\in\{0,1,2\}$ und $i$ gerade.
- $[(0, i), (1, i), (2, i)]$ ist Quer-M&uuml;hle, falls $i$ ungerade.

Variabelnamen in der Klasse `Game`:
- `stone`, `pos`: Tuple `(r, i)`, Position eines Spielsteins

In [4]:
#%%file muehle0.py
class Game:
    def __init__(self):
        self.players = (0, 1)
        self.muehlen = ([[(r, i), (r, i+1), (r, (i+2) % 8)] 
                         for r in range(3) for i in range(8) if self.is_even(i)
                        ] +
                        [[(0, i), (1, i), (2, i)] for i in range(8) if self.is_odd(i)]
                       )
        self.callback = lambda event, data: print(event, data)
        
    def new_game(self):
        self.board = {(r, i): None for r in range(3) for i in range(8)}
        self.ptm = 0 # player to move
        self.muehle = False  # hat jemand eine Muehle
        self.phase = 'place' # place, move
        self.callback('new_game', (self.muehlen,))
       
    def is_even(self, i):
        return i % 2 == 0
    
    def is_odd(self, i):
        return i % 2 == 1
    
    def is_muehle(self, stone):
        for muehle in self.muehlen:
            if stone in muehle and len(set(self.board[pos] for pos in muehle)) == 1:
                return True
          
    def move(self, player, tp, src, target=None):
        if tp == 'p':
            self.board[src] = player
            self.muehle = self.is_muehle(src)
            self.callback('place_stone', (src, player))
        elif tp == 'm':
            self.board[src] = None
            self.board[target] = player 
            self.muehle = self.is_muehle(target)
            self.callback('move_stone', (src, target, player))
        elif tp == 'r':
            self.board[src] = None
            self.muehle = False
            self.callback('remove_stone', (src,))
       
        if not self.muehle:
            self.ptm = 1 - self.ptm   
        else:
            print('muehle')
        
    def __repr__(self):
        return 'Am Zug: {}\nBoard {}'.\
                format(self.ptm, self.board)

### Klasse Game testen

In [ ]:
game = Game()
game.new_game()
game

In [ ]:
# wird Muehle erkannt
for i in range(3):
    game.move(0, 'p', (0, i))
game

In [ ]:
# Stein von Spieler 1 setzen
game.move(1, 'p', (0, 0))
game

In [ ]:
# wurden Muehlen korrekt berechnet?
game.muehlen